In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [11]:
!pip install tqdm

In [12]:
!pip install spotipy

In [13]:
import os

os.environ['SPOTIPY_CLIENT_ID'] = "933d94b754b44d8fb10c5067f1b17ef0"
os.environ['SPOTIPY_CLIENT_SECRET'] = "a04ebf35d69844af881b480ea040e863"

In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials()
)

In [15]:
sp.search(q="Kanye West", type="artist", limit=1)

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=Kanye%20West&type=artist',
  'limit': 1,
  'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=Kanye%20West&type=artist',
  'offset': 0,
  'previous': None,
  'total': 820,
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'followers': {'href': None, 'total': 31518836},
    'genres': ['rap'],
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb6e835a500e791bf9c27a422a',
      'height': 640,
      'width': 640},
     {'url': 'https://i.scdn.co/image/ab676161000051746e835a500e791bf9c27a422a',
      'height': 320,
      'width': 320},
     {'url': 'https://i.scdn.co/image/ab6761610000f1786e835a500e791bf9c27a422a',
      'height': 160,
      'width': 160}],
    'name': 'Kanye West',
    'popularity': 92,
    'type': 'artist',

In [16]:
kanye_id = "5K4W6rqBFWDnAN6FQUkS6x"

albums = sp.artist_albums(kanye_id, album_type="album", limit=50)
album_names = [(album["name"], album["id"]) for album in albums["items"]]
album_names[:10]

[('DONDA 2', '1ZkGNUz1un0b3Z7EsJl3ci'),
 ('VULTURES 2', '5RV2TNyjylqWJNxQyHBTeJ'),
 ('VULTURES 1', '4DOsPwJtokv6HEifZ6t5j6'),
 ('Donda (Deluxe)', '2Wiyo7LzdeBCsVZiRA6vVZ'),
 ('Donda', '5CnpZV3q5BcESefcB3WJmz'),
 ('JESUS IS KING', '0FgZKfoU2Br5sHOfvZKTI9'),
 ('KIDS SEE GHOSTS', '6pwuKxMUkNg673KETsXPUV'),
 ('ye', '2Ek1q2haOnxVqhvVKqMvJe'),
 ('The Life Of Pablo', '7gsWAHLeT0w7es6FofOXk1'),
 ('Yeezus', '7D2NdGvBHIavgLhmcwhluK')]

In [17]:
all_tracks = []

for album in albums["items"]:
    tracks = sp.album_tracks(album["id"])
    for t in tracks["items"]:
        all_tracks.append({
            "album": album["name"],
            "track_name": t["name"],
            "track_id": t["id"]
        })

In [18]:
df = pd.read_csv("kanye_all_albums_tracks.csv")
df.columns

Index(['album_title', 'artist', 'release_date', 'labels', 'genres',
       'critic_score', 'user_score', 'album_length', 'track_number',
       'track_title', 'track_length', 'featured_artists', 'track_rating',
       'track_rating_count', 'album_url'],
      dtype='object')

In [19]:
sp.track("7lIr3vVhpDkU5mQEDcnA0S")

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
  'available_markets': ['AR',
   'AU',
   'AT',
   'BE',
   'BO',
   'BR',
   'BG',
   'CA',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DK',
   'DO',
   'DE',
   'EC',
   'EE',
   'SV',
   'FI',
   'FR',
   'GR',
   'GT',
   'HN',
   'HK',
   'HU',
   'IS',
   'IE',
   'IT',
   'LV',
   'LT',
   'LU',
   'MY',
   'MT',
   'MX',
   'NL',
   'NZ',
   'NI',
   'NO',
   'PA',
   'PY',
   'PE',
   'PH',
   'PL',
   'PT',
   'SG',
   'SK',
   'ES',
   'SE',
   'CH',
   'TW',
   'TR',
   'UY',
   'US',
   'GB',
   'AD',
   'LI',
   'MC',
   'ID',
   'JP',
   'TH',
   'VN',
   'RO',
   'IL',
   'ZA',
   'SA',
   'AE',
   'BH',
   'QA',
   'OM',
 

In [20]:
sp.track("7lIr3vVhpDkU5mQEDcnA0S")["available_markets"][:10]

['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO']

In [23]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

tqdm.pandas()

def clean_track_title(title: str) -> str: # 1) нормализация названия трека для лучшего матчинга
    if pd.isna(title): # a) если пусто — вернём как есть
        return title

    t = str(title).strip()

    t = t.replace('"', "").replace("“", "").replace("”", "") # b) убираем лишние кавычки из csv

    t = re.sub(r"\s+", " ", t).strip() # c) убираем двойные пробелы

    return t

def safe_get_features(row): # 2) безопасный поиск трека + аудиофичи
    """
    row = строка df
    возвращает словарь с метаданными spotify + audio_features (или None)
    """

    track_name = clean_track_title(row.get("track_title"))
    artist = row.get("artist")
    featured = row.get("featured_artists")

    # a) если нет названия — нечего искать
    if pd.isna(track_name) or str(track_name).strip() == "":
        return None

    # b) набор стратегий поиска (по убыванию жёсткости)
    # 1) трек + артист
    # 2) трек без артиста (часто спасает для ¥$, KSG и т.п.)
    # 3) трек + первый featured артист (если есть)
    queries = []

    if pd.notna(artist) and str(artist).strip() != "":
        queries.append(f'track:"{track_name}" artist:"{artist}"')

    queries.append(f'track:"{track_name}"')  # fallback: только трек

    if pd.notna(featured) and str(featured).strip() != "": # если есть featured — попробуем по первому фиту
        first_feat = str(featured).split(",")[0].strip()
        if first_feat:
            queries.append(f'track:"{track_name}" artist:"{first_feat}"')

    track_item = None   # c) выполняем поиск по стратегиям
    used_query = None

    for q in queries:
        try:
            result = sp.search(q=q, type="track", limit=1)
        except Exception:
            continue

        items = result.get("tracks", {}).get("items", [])
        if len(items) > 0:
            track_item = items[0]
            used_query = q
            break

    if track_item is None: # d) если не нашли — вернём None (потом станет NaN в таблице)
        return {
            "spotify_found": 0,
            "spotify_query": None,
            "track_id": None,
            "track_uri": None,
            "popularity": None,
            "danceability": None,
            "energy": None,
            "valence": None,
            "tempo": None,
            "loudness": None,
            "speechiness": None,
            "instrumentalness": None,
            "liveness": None,
            "acousticness": None
        }

    track_id = track_item.get("id")
    track_uri = track_item.get("uri")
    popularity = track_item.get("popularity")

    features = None # e) аудиофичи (могут не вернуться -> тогда всё None)
    try:
        if track_id:
            features = sp.audio_features([track_id])[0]
    except Exception:
        features = None

    return {
        "spotify_found": 1,
        "spotify_query": used_query,
        "track_id": track_id,
        "track_uri": track_uri,
        "popularity": popularity,

        "danceability": features.get("danceability") if features else None,
        "energy": features.get("energy") if features else None,
        "valence": features.get("valence") if features else None,
        "tempo": features.get("tempo") if features else None,
        "loudness": features.get("loudness") if features else None,
        "speechiness": features.get("speechiness") if features else None,
        "instrumentalness": features.get("instrumentalness") if features else None,
        "liveness": features.get("liveness") if features else None,
        "acousticness": features.get("acousticness") if features else None
    }


# 3) применяем к нашему df (строка за строкой)
spotify_features = df.progress_apply(safe_get_features, axis=1)

# 4) превращаем список словарей в DataFrame
spotify_features_df = pd.DataFrame(list(spotify_features))

# 5) объединяем с исходным df
df_enriched = pd.concat([df.reset_index(drop=True), spotify_features_df.reset_index(drop=True)], axis=1)

df_enriched.to_csv("kanye_spotify_enriched.csv", index=False)

# 7) быстрая проверка качества матчинга
print("rows:", len(df_enriched))
print("spotify found %:", round(df_enriched["spotify_found"].mean() * 100, 2))
df_enriched.head()

  1%|          | 2/222 [00:00<00:15, 14.37it/s]ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0IW0qaeyxL5Et4UG2MrJKB with Params: {} returned 403 due to None
ERROR:spotipy.client:HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7wL7Lb8Q3aYyq6gmRL0PZq with Params: {} returned 403 due to None
100%|██████████| 222/222 [00:50<00:00,  4.38it/s]

rows: 222
spotify found %: 99.55


,album_title,artist,release_date,labels,genres,critic_score,user_score,album_length,track_number,track_title,...,popularity,danceability,energy,valence,tempo,loudness,speechiness,instrumentalness,liveness,acousticness
0,The College Dropout,Kanye West,"February10,2004/ Release Date","Def Jam, Roc-A-Fella","Chipmunk Soul, Pop Rap, Conscious Hip Hop, Neo...",81,91,"1 hour, 16 minutes",1,Intro,...,14.0,None,None,None,None,None,None,None,None,None
1,The College Dropout,Kanye West,"February10,2004/ Release Date","Def Jam, Roc-A-Fella","Chipmunk Soul, Pop Rap, Conscious Hip Hop, Neo...",81,91,"1 hour, 16 minutes",2,We Don't Care,...,67.0,None,None,None,None,None,None,None,None,None
2,The College Dropout,Kanye West,"February10,2004/ Release Date","Def Jam, Roc-A-Fella","Chipmunk Soul, Pop Rap, Conscious Hip Hop, Neo...",81,91,"1 hour, 16 minutes",3,Graduation Day,...,55.0,None,None,None,None,None,None,None,None,None
3,The College Dropout,Kanye West,"February10,2004/ Release Date","Def Jam, Roc-A-Fella","Chipmunk Soul, Pop Rap, Conscious Hip Hop, Neo...",81,91,"1 hour, 16 minutes",4,All Falls Down,...,84.0,None,None,None,None,None,None,None,None,None
4,The College Dropout,Kanye West,"February10,2004/ Release Date","Def Jam, Roc-A-Fella","Chipmunk Soul, Pop Rap, Conscious Hip Hop, Neo...",81,91,"1 hour, 16 minutes",5,I'll Fly Away,...,55.0,None,None,None,None,None,None,None,None,None
